# NFL Punt Analysis Rule Change Analysis
## Kevin McGovern and Todd Steussie
This notebook contains visualizations used to evaluate NFL punt plays in 2016 and 2017 and support a suggested rule change.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from ipywidgets import interactive
from IPython.display import HTML

from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import CustomJS, Slider, Button, BoxAnnotation, LabelSet, LinearAxis, Range1d
from bokeh.layouts import row, widgetbox
import bokeh.models as bmo

import seaborn as sns; sns.set()

import matplotlib.pyplot as plt

output_notebook()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input/nfl-punt-analysis-data-output"))

# Load full data set
puntDF = pd.read_csv('../input/nfl-punt-analysis-data-output/playerMvmt-level-data.csv')
playLvlDF = pd.read_csv('../input/nfl-punt-analysis-data-output/play-level-data.csv')
playLvlCleanDF = playLvlDF[playLvlDF['Primary_Impact_Type'] != 'Unclear']

In [ ]:
def get_data_over_time(playID = 3746, actors = ['27654.0','33127.0']):
    limitedSample = puntDF.loc[(puntDF['PlayID'] == playID)]
    limitedSample = limitedSample[limitedSample['GSISID_y'].isin(actors)]
    return limitedSample
def update_sample(playID = 3746, timeNum = 1):
    limitedSample = puntDF.loc[(puntDF['PlayID'] == playID) & (puntDF['TimeNum'] == timeNum)]
    return limitedSample

In [ ]:
def update_interactors(limitedSample = puntDF.loc[(puntDF['PlayID'] == 3746)]):
    interactors = limitedSample[(limitedSample['GSISID_x'] == limitedSample['GSISID_y']) | (limitedSample['Primary_Partner_GSISID'] == limitedSample['GSISID_y'])]
    return interactors

def get_play(playID = 3746):
    return playLvlDF.loc[(playLvlDF['PlayID'] == playID)]

def generatePlayText(playID = 3746):
    playCalled = get_play(playID)
    stmt = str(playCalled['GSISID'].iloc[0]) + ' was injured in ' + str(playCalled['Season_Year'].iloc[0]) + '. The contact was made by ' + playCalled['Player_Activity_Derived'].iloc[0] + ' on ' + playCalled['Primary_Impact_Type'].iloc[0] + ' contact.'
    return stmt

In [ ]:
def update_plot(timeNum, playID):
    #update circles
    limitedSample = update_sample(playID, timeNum)
    x = limitedSample['x']
    y = limitedSample['y']
    colorMap = ['#ff0000' if x in(limitedSample['GSISID_x'].values) else '#0000ff' if x in(limitedSample['Primary_Partner_GSISID'].values) else '#545454' for x in limitedSample.GSISID_y]

    circlePlt.data_source.data['x'] = x
    circlePlt.data_source.data['y'] = y
    circlePlt.data_source.data['fill_color'] = colorMap
    
    #update lines
    distanceLine = update_interactors(limitedSample)
    line_x = distanceLine['x']
    line_y = distanceLine['y']
    lines.data_source.data['x'] = line_x
    lines.data_source.data['y'] = line_y
    #plt.data_source.data['source'] = source
    
    #labels = LabelSet(x=x, y=y, text='gsisid', level='glyph',
    #           x_offset=5, y_offset=5, render_mode='canvas')
    #fig.renderers.pop()
    #fig.add_layout(labels)
    push_notebook(handle=bokeh_handle)
    
    # Update distance chart
    distanceDF = get_data_over_time(playID, actors = [get_play(playID)['GSISID'].iloc[0]])
    x = distanceDF['TimeNum']
    y = distanceDF['Distance']

    distanceLines.data_source.data['x'] = x
    distanceLines.data_source.data['y'] = y
    
    push_notebook(handle=bokeh_handle2)
    
    # Update speed chart
    distanceDF = get_data_over_time(playID, actors = [get_play(playID)['GSISID'].iloc[0]])
    x = distanceDF['TimeNum']
    y = distanceDF['Speed']
    injSpeedlines.data_source.data['x'] = x
    injSpeedlines.data_source.data['y'] = y

    distanceDF = get_data_over_time(playID, actors = [get_play(playID)['Primary_Partner_GSISID'].iloc[0]])
    x = distanceDF['TimeNum']
    y = distanceDF['Speed']
    primActorSpeedlines.data_source.data['x'] = x
    primActorSpeedlines.data_source.data['y'] = y
    
    push_notebook(handle=bokeh_handle3)
    
    return ""#generatePlayText(playID)

In [ ]:
# Create function for play charting
def create_bokeh_play_x_y_lines(playID, actors, figure):

    # calculate base data set
    limitedSample = get_data_over_time(playID = playID, actors = actors)
    x = limitedSample[limitedSample['GSISID_y'] == float(actors[0])]['x']
    y = limitedSample[limitedSample['GSISID_y'] == float(actors[0])]['y']
    x2 = limitedSample[limitedSample['GSISID_y'] == float(actors[1])]['x']
    y2 = limitedSample[limitedSample['GSISID_y'] == float(actors[1])]['y']

    # add line renderers with color, width, and alpha
    figure.line(x, y, line_color='#b22222',line_width=3, alpha=0.8)
    figure.line(x2, y2, line_color='#4682b4',line_width=3, alpha=0.8)
    #plt2 = figure.circle(x, y, size=8, fill_color=color_map, alpha=0.8)

    low_box = BoxAnnotation(right=10, fill_alpha=0.2, fill_color='#555555')
    mid_box = BoxAnnotation(left=10, right=110, fill_alpha=0.2, fill_color='green')
    high_box = BoxAnnotation(left=110, fill_alpha=0.2, fill_color='#555555')

    figure.line([10,10,20,20,30,30,40,40,50,50,60,60,70,70,80,80,90,90,
              100,100,110,110,120,120],
             [55.5,0,0,55.5,55.5,0,0,55.5,55.5,0,0,55.5,55.5,0,0,55.5,
             55.5,0,0,55.5,55.5,0,0,0], line_color='grey')

    figure.add_layout(low_box)
    figure.add_layout(mid_box)
    figure.add_layout(high_box)

    return figure

# Create function to handle line chart creation
def create_bokeh_lines(playID, actor, X, Y, lineFig, color='black', rightAxis=None, rightAxisName=""):
    # calculate base data set
    distanceDF = get_data_over_time(playID = playID, actors = [actor])
    distanceDF['TimeFloor'] = distanceDF['TimeNum']//5
    distanceDF = distanceDF.groupby(['TimeFloor','PlayID'], as_index=False)[Y].mean()
    distanceDF['TimeNum'] = distanceDF['TimeFloor']*5
    x = distanceDF[X]
    y = distanceDF[Y]

    # return the line renderer with a color, width, and alpha
    if rightAxis==None:
        return lineFig.line(x, y, line_color = color, line_width = 2, alpha = 0.8)
    else:
        # Setting the second y axis range name and range
        minval = y.min()*.8 if y.min()>0 else -y.max()*.05 if y.min()==0 else y.min()*1.2
        lineFig.extra_y_ranges = {rightAxis: Range1d(minval,y.max()*1.2)}
        lineFig.add_layout(LinearAxis(y_range_name=rightAxis, axis_label=Y + ' ('+ rightAxisName +')'), 'right')
        return lineFig.line(x, y, line_color = color, line_width = 2, alpha = 0.8, y_range_name=rightAxis)

In [ ]:
# Create figure plot
distanceLineFig = figure(plot_width=800, plot_height=400)
# Create distance lines
distanceLines = create_bokeh_lines(3746, '27654.0', 'TimeNum', 'Distance', distanceLineFig)

# Create injured player and primary actor speed charts
speedLineFig = figure(plot_width=800, plot_height=400)
injSpeedlines = create_bokeh_lines(3746, '27654.0', 'TimeNum', 'Speed', speedLineFig, 'red')
primActorSpeedlines = create_bokeh_lines(3746, '33127.0', 'TimeNum', 'Speed', speedLineFig, 'blue')

# Play Analysis
Before we decided on a rule change, we reviewed each play individually and looked the cause of the concussion. Below is our analysis that lead to our suggested rule change.

Note: we removed the one injury for this analysis where the cause was 'unclear', as it did not inform the high-level analysis without a clear cause.

The breakdown of primary impact shows that the vast marjority of the concussions in 2016 and 2017 occurred on plays where the contact was made on a helmet-to-body or helmet-to-helmet collision

In [ ]:
## Primary Impact
plt.figure(figsize=(10,5))
sns.set_style("darkgrid")
sns.set_palette("YlGnBu", n_colors=3)
countplt = sns.countplot(x="Primary_Impact_Type", data=playLvlCleanDF)

## Of the plays evaluated, a significant portion of concussions occurred on tackling plays with Helmet-to-body contact

In [ ]:
uniform_data = pd.pivot_table(playLvlCleanDF, values='PlayID', index=['Player_Activity_Derived'], columns=['Primary_Impact_Type'], aggfunc='count')
plt.figure(figsize=(10,8))
ax = sns.heatmap(uniform_data, annot=True, cmap="YlGnBu")

In [ ]:
## Friendly fire chart

uniform_data = pd.pivot_table(playLvlCleanDF, values='PlayID', index=['Friendly_Fire'], columns=['Primary_Impact_Type'], aggfunc='count')
plt.figure(figsize=(10,8))
ax = sns.heatmap(uniform_data, annot=True, cmap="YlGnBu")

### Some trends start to emerge but it wasn't until we looked at game film that we saw a theme.


Game analysis showed what wasn't immediately clear from the data. There were common characteristics in plays where players either attempted a low block or attempted a diving or lower-body tackle.

## Of the 8 plays that lead to a concussion from blocking in 2016 and 2017, 1 injury was caused by the player attempting to block the impacting player below the waist

## Additionally, of the 10 plays that lead to a concussion from tackling in 2016 and 2017, 4 injuries occurred when the injured player attempted to dive or tackle another player below the waist

We concede that these are not high numbers, however, there was a common thread among these examples, where the blocks and tackles above the waist did not share observable common characteristics.

In [ ]:
## Chart supporting above statement

In [ ]:
HTML('<video width="600" height="400" controls> <source src="http://a.video.nfl.com//films/vodzilla/153245/Punt_by_Brad_Nortman-QiQqjFdU-20181119_155917392_5000k.mp4" type="video/mp4"></video>')

# Suggested Rule Change
## In order to reduce the risk of injury, we recommend two rule changes to limit contact below the waist on punt plays
## Rule 1: Penalize blocks below the waist
The receiving and kicking teams should both be restricted from attempting any block that results in their head going below the waist of another player. Intentionally going to the ground in an attempt to make a block puts the player at risk of suffering blows to the head.

## Rule 2: Penalize diving or lower-body tackles
The kicking team should be restricted from attempting to tackle the ball-carrier with either a diving tackle or a tackle that hits the lower body of the ball-carrier. Players suffered inadvertent contact when attempting to make a tackle by either hitting their head on the body of a teammate or opposing player. 

The rules should be implemented separately because it is possible that the second rule could potentially lead to more helmet-to-helmet collisions. A reduction in tackles below the waist could lead to more high-body hits, with players accidentally hitting the head of the ball-carrier or other players. This rule should be tested in pre-season before attempting to implement in the regular season. However, due to the number plays affected by diving tackles in this small sample, a diving or lower body tackling rule should still be enforced.


## Rule Change #1 Example
### JAX-HOU 2016 Week 15 example of low block on incoming player leads to a concussion of blocking player
The charts below show how in a 2016 week 15 Jaguars vs. Texans game, a player suffered a concussion while diving at an incoming player.

This exemplifies the plays that occur where players are not necessarily moving fast, but are still at risk for concussions due to diving in an attempt to block another player.

In [ ]:
# Distance vs speed
LineFigJAX2016_15 = figure(plot_width=800, plot_height=400, title="JAX-HOU Distance Between Impacted Players (blue) vs. Speed of Injured Player (red)")
# Create distance lines
LineFigJAX2016_15Rtn = create_bokeh_lines(2918, '32120.0', 'TimeNum', 'Distance', LineFigJAX2016_15, color='#008ac2')
LineFigJAX2016_15.yaxis.axis_label = "Distance (yards)"
LineFigJAX2016_15.xaxis.axis_label = "Play Time (milliseconds)"
LineFigJAX2016_15.background_fill_color = "#efefef"
LineFigJAX2016_15.background_fill_alpha = 0.5
#LineFigRaiders2017_05.legend.location = "top_right"

LineFigJAX2016_15Rtn2 = create_bokeh_lines(2918, '32120.0', 'TimeNum', 'Speed', LineFigJAX2016_15, color='#de425b', rightAxis='rightRange', rightAxisName='red')

LineFigJAX2016_15show = show(row(LineFigJAX2016_15), notebook_handle=True)

In the chart above, we compared speed of the injured player and distance between the injured player and the impacting player.  The two players are never more than 5 yards apart from the start of the play (roughly 60 millisecond mark) until the point of contact (roughly 150 milliseconds). The speed of the player alternates between positive and negative speed until he dives at the legs of the incoming player at roughly 120 millisecon mark, quickly making contact with the impacting  player.

In [ ]:
# Speed vs acceleration
# Create injured player and primary actor speed charts
speedLineFigJAX2016_15 = figure(plot_width=800, plot_height=400, title="JAX-HOU Speed (red) vs. Acceleration (green) of Injured Player")
injSpeedlinesRJAX2016_15 = create_bokeh_lines(2918, '32120.0', 'TimeNum', 'Speed', speedLineFigJAX2016_15, color='#de425b')
speedLineFigJAX2016_15.yaxis.axis_label = "Speed"
speedLineFigJAX2016_15.xaxis.axis_label = "Play Time (milliseconds)"
speedLineFigJAX2016_15.background_fill_color = "#efefef"
speedLineFigJAX2016_15.background_fill_alpha = 0.5

accelSpeedlinesRJAX2016_15 = create_bokeh_lines(2918, '32120.0', 'TimeNum', 'Acceleration', speedLineFigJAX2016_15, color='#00FA9A', rightAxis='rightRange', rightAxisName='green')

JAX2016_15show = show(row(speedLineFigJAX2016_15), notebook_handle=True)

In the chart above we compare the speed of the injured player against the acceleration of the injured player. As the play begins the injured player begins to accelerate until the point of contact (roughly at 150 milliseconds). After the contact there is another period of acceleration and increase in speed that is actually the player going backwards. Overall, this is neither a high speed nor a high acceleration injury but it is totally caused by the player diving at the legs of the impacting player. 

In [ ]:
# Paths
XYFigJAX2016_15 = figure(plot_width=800, plot_height=400, x_range=(0,120), y_range=(0, 53.3), title="JAX-HOU Path of Injured (red) and Impacting (blue) Players")
XYFigJAX2016_15upd = create_bokeh_play_x_y_lines(playID = 2918, actors = ['32120.0', '32725.0'], figure = XYFigJAX2016_15)
bokeh_handle6 = show(row(XYFigJAX2016_15upd), notebook_handle=True)

The above chart shows the path of the injured player (red) compared with the path of the impacting player (blue). The initial contact occurred early in the play near the line where a low-block occurred.

In [ ]:
HTML('<iframe width="560" height="315" src="https://streamable.com/s/6lg2y/xrywsg.mp4" frameborder="0" allowfullscreen></iframe>')

View the video above to see the exact contact that led to the concussion.

## Rule Change #2 Example
### BAL-OAK 2017 Week 5 example of low dive tackle leading to contact with teammate

In the below example a Raiders player attempts a diving tackle against the ball-carrier. He does not make contact with the ball-carrier, instead his head inadvertently hits the body of his teammate.

In [ ]:
HTML('<video width="560" height="315" controls> <source src="http://a.video.nfl.com//films/vodzilla/153273/King_62_yard_punt-BSOws7nQ-20181119_165306255_5000k.mp4" type="video/mp4"></video>')

In [ ]:


# Distance vs speed
LineFigRaiders2017_05 = figure(plot_width=800, plot_height=400, title="BAL-OAK Distance Between Impacted Players (blue) vs. Speed of Injured Player (red)")
# Create distance lines
LineFigRaiders2017_05Rtn = create_bokeh_lines(2072, '29492.0', 'TimeNum', 'Distance', LineFigRaiders2017_05, color='#008ac2')
LineFigRaiders2017_05.yaxis.axis_label = "Distance (yards)"
LineFigRaiders2017_05.xaxis.axis_label = "Play Time (milliseconds)"
LineFigRaiders2017_05.background_fill_color = "#efefef"
LineFigRaiders2017_05.background_fill_alpha = 0.5
#LineFigRaiders2017_05.legend.location = "top_right"

LineFigRaiders2017_05Rtn2 = create_bokeh_lines(2072, '29492.0', 'TimeNum', 'Speed', LineFigRaiders2017_05, color='#de425b', rightAxis='rightRange', rightAxisName='red')

LineFigRaiders2017_05show = show(row(LineFigRaiders2017_05), notebook_handle=True)

The above chart shows that both distance between the injured and impacting player and the speed of the injured player are maximized at roughtly the same time. The moment of impact occurs around the 230-240 millisecond mark. As expected, the speed of the injured player quickly drops when the two players collide.

In [ ]:


# Speed vs acceleration
# Create injured player and primary actor speed charts
speedLineFigRaiders2017_05 = figure(plot_width=800, plot_height=400, title="BAL-OAK Speed (red) vs. Acceleration (green) of Injured Player")
injSpeedlinesRaiders2017_05 = create_bokeh_lines(2072, '29492.0', 'TimeNum', 'Speed', speedLineFigRaiders2017_05, color='#de425b')
speedLineFigRaiders2017_05.yaxis.axis_label = "Speed"
speedLineFigRaiders2017_05.xaxis.axis_label = "Play Time (milliseconds)"
speedLineFigRaiders2017_05.background_fill_color = "#efefef"
speedLineFigRaiders2017_05.background_fill_alpha = 0.5

accelSpeedlinesRaiders2017_05 = create_bokeh_lines(2072, '29492.0', 'TimeNum', 'Acceleration', speedLineFigRaiders2017_05, color='#00FA9A', rightAxis='rightRange', rightAxisName='green')

Raiders2017_05show = show(row(speedLineFigRaiders2017_05), notebook_handle=True)

The above chart shows speed and acceleration compared for the injured player. Acceleration increases rapidly when the ball is kicked and that speed is sustained while running done field. At the moment of impact (230-240) the player quickly decelerates as he quickly comes to a stop after diving and running into his teammate. Because the player dove while running at such a high speed, the unintentional impact had a significant effect.

In [ ]:
## Create X-Y Chart showing path of impacted players

XYFigRaiders2017_05 = figure(plot_width=800, plot_height=400, x_range=(0,120), y_range=(0, 53.3), title="OAK-BAL Path of Injured (red) and Impacting (blue) Players")
XYFigRaiders2017_05 = create_bokeh_play_x_y_lines(playID = 2072, actors = ['29492.0', '33445.0'], figure = XYFigRaiders2017_05)
bokeh_handle4 = show(row(XYFigRaiders2017_05), notebook_handle=True)

In [ ]:
HTML('<iframe width="560" height="315" src="https://streamable.com/s/n5e7l/imupug.mp4" frameborder="0" allowfullscreen></iframe>')

## Recap
The two examples above illustrate particular situations where a low block and a low dive resulted in concussions. It is our belief that these concussions could have been avoided had the players not gone to the ground on the play. 


One note of caution, because the data we evaluated only used the 2016 and 2017 seasons, it is possible that this sample size is over-representative. To fully test this theory, additional seasons would have to be evaluated or the rule should be heavily monitored to see if it leads to a decrease in concussions on punt plays

## Interactive Section
The charts below allow the selection of other plays to navigate through time and view different results. The controls to change inputs are located immediately following the charts.

### Please note: to run this workbook with fully interactivity, you must first fork the notebook and run all cells. The last cells has interactive elements for PlayID and Time during play 



### Distance between injured player and primary actor

In [ ]:
bokeh_handle2 = show(row(distanceLineFig), notebook_handle=True)

### Speed of injured player (red) vs. primary actor (blue)

In [ ]:
bokeh_handle3 = show(row(speedLineFig), notebook_handle=True)

### Play Motion Chart

In [ ]:
# Create chart for plays
# calculate base data set
limitedSample = update_sample(playID = 3746)
x = limitedSample['x']
y = limitedSample['y']

# get data for distance line
distanceLine = update_interactors(limitedSample)
line_x = distanceLine['x']
line_y = distanceLine['y']


fig = figure(plot_width=800, plot_height=400, x_range=(0,120), y_range=(0, 53.3))

color_map = ['#ff0000' if x in(limitedSample['GSISID_x'].values) else '#0000ff' if x in(limitedSample['Primary_Partner_GSISID'].values) else '#545454' for x in limitedSample.GSISID_y[limitedSample['TimeNum'] == 0]]
#print(color_map)
dfDict = {
        'id': limitedSample.loc[:,'GSISID_y'],
        'x': x,
        'y': y,
        'color': color_map
    }
#print(pd.DataFrame(dfDict))

source = ColumnDataSource(data=dict(x=x,
                                    y=y,
                                    gsisid=limitedSample['GSISID_y'],
                                   colors=color_map))
#labels = LabelSet(x='x', y='y', text='gsisid', level='glyph',
#              x_offset=5, y_offset=5, source=source, render_mode='canvas')

# add a circle renderer with a size, color, and alpha
lines = fig.line(line_x, line_y, line_color='black',line_width=2, alpha=0.6)
circlePlt = fig.circle(x, y, size=8, fill_color=color_map, alpha=0.8)
#plt = fig.circle(x='x', y='y', size=8, color='colors', alpha=0.6, source=source)



low_box = BoxAnnotation(right=10, fill_alpha=0.2, fill_color='#555555')
mid_box = BoxAnnotation(left=10, right=110, fill_alpha=0.2, fill_color='green')
high_box = BoxAnnotation(left=110, fill_alpha=0.2, fill_color='#555555')

fig.line([10,10,20,20,30,30,40,40,50,50,60,60,70,70,80,80,90,90,
          100,100,110,110,120,120],
         [55.5,0,0,55.5,55.5,0,0,55.5,55.5,0,0,55.5,55.5,0,0,55.5,
         55.5,0,0,55.5,55.5,0,0,0], line_color='grey')

fig.add_layout(low_box)
fig.add_layout(mid_box)
fig.add_layout(high_box)
#fig.add_layout(labels)

bokeh_handle = show(row(fig), notebook_handle=True)

### Change the time or play viewed above

In [ ]:

interactive(update_plot, playID = playLvlDF[playLvlDF['InjCtrlFlag'] == 'Injury']['PlayID'].sort_values().values, timeNum = (0,350))